# С помощью этой методики можно найти свободные IMSI диапазоны, а также создать файл с ключами для отправки кастомеру

In [1]:
# Загрузить данные из файлов
#----------------------------------

import pandas as pd
from pandas import DataFrame
from os.path import join, normpath
import datetime as dt

downloads = './DATA/SoftProfiles/Data/'
result_file = 'result_file.csv'
files = ['rb_42501_25_06_20181.txt', 'rb_42501_2_09_20181.txt', 'rb_26006_25_06_20181.txt', 'rb_26006_2_09_20181.txt']

reseller_name = 'Simlocal'
s1_account_name = 'Test SIM cards'
s2_account_name = ''

customer_name = 'Simlocal'
current_date = dt.datetime.now().strftime('%y%m%d')
file_s1_name = f'S1_{customer_name}_{current_date}.txt'
file_s2_name = f'S2_{customer_name}_{current_date}.txt'

df_imsi = DataFrame(columns = ['IMSI', 'KI', 'OPC'])

for file in files:
    df_imsi = df_imsi.append(pd.read_table(join(downloads, file), sep=' ',header=0, engine='python'))

df_imsi.IMSI = df_imsi.IMSI.astype('int64')
df_imsi.sort_values(by = 'IMSI', inplace = True)

df_imsi.head()

,IMSI,KI,OPC
0,260060140890000,1AEA5D122A1F2476EBD9350F10C0EFDD,5EE04A60E740AF9890C397A899109B25
1,260060140890001,09B595FF54EEFE9E547237B4D7499202,9274F199697E5775100367CAE65B24D0
2,260060140890002,744A3D38AFBCBB0292FF9807D21E8B4D,8D7B5100C7BF361382490BFA632F8A55
3,260060140890003,2244EDED489D62B569EC8179FC73489F,2D63E23033BB91337C8DD03362984B44
4,260060140890004,FCB72503EA93EEB2A36CFBC23256697D,7697EA5B130950C519EDA1487C50CE2F


In [2]:
# Предварительная проверка диапазонов (опционально)
#----------------------------------

list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]

df = df_imsi.groupby(df_imsi.IMSI.astype('str').str.slice(0,12))['IMSI'].agg(list_agg).reset_index()
df.IMSI = df.IMSI.astype('int64')
df.rename(columns = {'IMSI':'IMSI_RANGE'}, inplace = True)

df['imsi_shifted'] = df['IMSI_RANGE'].shift(-1)
df.fillna(0,inplace=True)
df['imsi_shifted'] = df['imsi_shifted'].astype('int64')

def check_imsi(x):
    x['imsi_num'] = x['IMSI_MAX'] - x['IMSI_MIN'] + 1
    x['check_range'] = x['imsi_shifted'] - x['IMSI_RANGE']
    if x['NUM'] == (x['IMSI_MAX'] - x['IMSI_MIN'] + 1):
        x['imsi_num_check'] = 'OK'
    else:
        x['imsi_num_check'] = 'NOK'
    return x

df = df.apply(check_imsi, axis=1)
#df

In [3]:
df

,IMSI_RANGE,IMSI_MIN,IMSI_MAX,NUM,imsi_shifted,imsi_num,check_range,imsi_num_check
0,260060140890,260060140890000,260060140890999,1000,260060140891,1000,1,OK
1,260060140891,260060140891000,260060140891999,1000,260060140892,1000,1,OK
2,260060140892,260060140892000,260060140892999,1000,260060140893,1000,1,OK
3,260060140893,260060140893000,260060140893999,1000,260060140894,1000,1,OK
4,260060140894,260060140894000,260060140894999,1000,260060140895,1000,1,OK
5,260060140895,260060140895000,260060140895999,1000,260060140896,1000,1,OK
6,260060140896,260060140896000,260060140896999,1000,260060140897,1000,1,OK
7,260060140897,260060140897000,260060140897999,1000,260060140898,1000,1,OK
8,260060140898,260060140898000,260060140898999,1000,260060140899,1000,1,OK
9,260060140899,260060140899000,260060140899999,1000,260060149991,1000,9092,OK


In [7]:
df.to_csv(join(downloads,'soft_ranges.csv'), index=False)

In [3]:
# Из базы данных OCS выгружаем IMSI
#----------------------------------

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT
r.RESELLER_NAME
,a.ACCOUNT_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
AND ss.status = 'Active'
AND
(si.IMSI BETWEEN '260060140890000' AND '260060140899999'
OR si.IMSI BETWEEN  '260060149991100' AND '260060149999999'
OR si.IMSI BETWEEN  '425019613998060' AND '425019613999999'
OR si.IMSI BETWEEN  '425019629920000' AND '425019629939999')
'''

df_ocs_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_imsi['IMSI'] = df_ocs_imsi['IMSI'].astype('int64')
cnxn.close()
df_ocs_imsi.head()

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI
0,Acloud RT,Acloud Test,Active,260060149991108
1,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991111
2,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991112
3,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991113
4,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991114


In [4]:
# Объединяем IMSI с ключами с данными OCS
#----------------------------------

df_ocs_imsi_ki = pd.merge(df_imsi, df_ocs_imsi, how='left', on='IMSI')
list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]
list_col = ['RESELLER_NAME', 'ACCOUNT_NAME', 'SUB_STATUS', df_ocs_imsi_ki.IMSI.astype('str').str.slice(0,12)]
df_ocs_imsi_ki_group = df_ocs_imsi_ki.groupby(list_col)['IMSI'].agg(list_agg).reset_index().sort_values(by='IMSI')

df_ocs_imsi_ki_group.head()

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
10,MCN Telecom,MCN Telecom Soft SIM,Active,260060140890,260060140890000,260060140890999,1000
11,MCN Telecom,MCN Telecom Soft SIM,Active,260060140891,260060140891000,260060140891999,1000
12,MCN Telecom,MCN Telecom Soft SIM,Active,260060140892,260060140892000,260060140892999,1000
13,MCN Telecom,MCN Telecom Soft SIM,Active,260060140893,260060140893000,260060140893999,1000
14,MCN Telecom,MCN Telecom Soft SIM,Active,260060140894,260060140894000,260060140894999,1000


In [8]:
# For test accounts to use:
# 425019613998
# 260060149991
df_ocs_imsi_ki_group[df_ocs_imsi_ki_group['IMSI'] == '260060149991'].sort_values('IMSI_MIN')

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
4,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,260060149991,260060149991100,260060149991104,5
14,MCN Telecom,MCN Telecom Soft SIM,Active,260060149991,260060149991105,260060149991107,3
0,Acloud RT,Acloud Test,Active,260060149991,260060149991108,260060149991109,2
2,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991,260060149991110,260060149991122,12
50,Tapmobile RT,Tapmobile Test Sim cards,Active,260060149991,260060149991116,260060149991116,1
59,Watchdata SG,Watchdata P4 Soft test,Active,260060149991,260060149991123,260060149991123,1
24,Roamability Invertory,P4 Soft simcards inventory,Active,260060149991,260060149991124,260060149991996,873
48,Taisys,Test Sim cards,Active,260060149991,260060149991997,260060149991999,3


In [12]:
df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').str.startswith('26006014999199', na=False)].sort_values('IMSI')

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI
13920,Roamability Invertory,P4 Soft simcards inventory,Active,260060149991990
13921,Roamability Invertory,P4 Soft simcards inventory,Active,260060149991991
13922,Roamability Invertory,P4 Soft simcards inventory,Active,260060149991992
13923,Roamability Invertory,P4 Soft simcards inventory,Active,260060149991993
13924,Roamability Invertory,P4 Soft simcards inventory,Active,260060149991994
42,Joy telecom,Joy telecom P4 Soft test,Active,260060149991995
43,Joy telecom,Joy telecom P4 Soft test,Active,260060149991996
36820,Taisys,Test Sim cards,Active,260060149991997
36821,Taisys,Test Sim cards,Active,260060149991998
36822,Taisys,Test Sim cards,Active,260060149991999


In [6]:
df_ocs_imsi_ki_group[df_ocs_imsi_ki_group['RESELLER_NAME'] == reseller_name]

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
49,Simlocal,Test SIM cards,Active,260060149991,260060149991994,260060149991994,1
50,Simlocal,Test SIM cards,Active,425019613998,425019613998996,425019613998996,1


In [7]:
# Выгружаем файлы с ключами
#----------------------------------

if len(s1_account_name) != 0:
    df_ocs_imsi_ki.loc[(df_ocs_imsi_ki.RESELLER_NAME == reseller_name) & (df_ocs_imsi_ki.ACCOUNT_NAME == s1_account_name),
                   ['IMSI', 'KI', 'OPC']].to_csv(join(downloads, file_s1_name), index=False)

if len(s2_account_name) != 0:
    df_ocs_imsi_ki.loc[(df_ocs_imsi_ki.RESELLER_NAME == reseller_name) & (df_ocs_imsi_ki.ACCOUNT_NAME == s2_account_name),
                   ['IMSI', 'KI', 'OPC']].to_csv(join(downloads, file_s2_name), index=False)